In [1]:
import polars as pl 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pl.read_parquet('../sentiment_analysis_2.parquet')
df.head()

user_id,item_id,sentiment_analysis_2,recommend,__index_level_0__
str,i32,i32,i32,i64
"""76561197970982…",1250,2,1,0
"""76561197970982…",22200,2,1,1
"""76561197970982…",43110,2,1,2
"""js41637""",251610,2,1,3
"""js41637""",227300,2,1,4


In [3]:
df= df.drop(columns=['recommend', '__index_level_0__' ])
df = df.rename({'sentiment_analysis_2': 'rating'})
df.head()

user_id,item_id,rating
str,i32,i32
"""76561197970982…",1250,2
"""76561197970982…",22200,2
"""76561197970982…",43110,2
"""js41637""",251610,2
"""js41637""",227300,2


In [4]:
df_pivot = df.pivot(index='user_id', columns='item_id', values='rating')
df_pivot.head()

user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,…,210170,319150,225833,378660,242590,339910,61730,259410,318520,319970,378580,420930,349740,254130,274350,381020,358260,358340,358290,256500,431840,385890,203250,348670,385630,385980,369000,294230,367780,305920,306040,414390,307130,209120,220090,262850,431510
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""76561197970982…",2,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""js41637""",null,null,null,2,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""evcentric""",null,null,null,null,null,null,2,2,2,0,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""doctr""",null,null,null,null,null,null,null,null,null,null,null,null,2,2,2,2,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""maplemage""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,2,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [5]:
users = pl.read_parquet('../CleanData/users_items.parquet')
users.head()

user_id,playtime_forever,playtime_2weeks,item_id,item_name,__index_level_0__
str,i64,i64,i32,str,i64
"""76561197970982…",6,0,10,"""Counter-Strike…",0
"""76561197970982…",0,0,20,"""Team Fortress …",1
"""76561197970982…",7,0,30,"""Day of Defeat""",2
"""76561197970982…",0,0,40,"""Deathmatch Cla…",3
"""76561197970982…",0,0,50,"""Half-Life: Opp…",4


In [6]:
total_users = users['user_id']

In [7]:
users_rev = df_pivot['user_id']

In [8]:
len(users_rev)

25458

In [9]:
users_append = pl.concat([total_users, users_rev], how= 'vertical')
users_append.head()

user_id
str
"""76561197970982…"
"""76561197970982…"
"""76561197970982…"
"""76561197970982…"
"""76561197970982…"
"""76561197970982…"
"""76561197970982…"
"""76561197970982…"
"""76561197970982…"


In [10]:
len(users_append)

5119563

In [11]:
users_append = users_append.unique()

In [12]:
len(users_append)

73757

In [13]:
reviewed_games = list(df_pivot.columns)
reviewed_games.remove('user_id')

In [14]:
df_append = pl.DataFrame({'user_id': users_append})

In [15]:
for game in reviewed_games:
    df_append = df_append.with_columns(['user_id', pl.lit(None).alias(game)])

In [16]:
df_append.head()

user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,…,210170,319150,225833,378660,242590,339910,61730,259410,318520,319970,378580,420930,349740,254130,274350,381020,358260,358340,358290,256500,431840,385890,203250,348670,385630,385980,369000,294230,367780,305920,306040,414390,307130,209120,220090,262850,431510
str,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""76561198086466…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""pooldeadlol""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""housty8""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""76561198087476…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""OmGwTfLoLrOfLm…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [17]:
df_pivot = pl.concat([df_pivot, df_append], how= 'vertical_relaxed')
df_pivot.head()

user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,…,210170,319150,225833,378660,242590,339910,61730,259410,318520,319970,378580,420930,349740,254130,274350,381020,358260,358340,358290,256500,431840,385890,203250,348670,385630,385980,369000,294230,367780,305920,306040,414390,307130,209120,220090,262850,431510
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""76561197970982…",2,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""js41637""",null,null,null,2,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""evcentric""",null,null,null,null,null,null,2,2,2,0,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""doctr""",null,null,null,null,null,null,null,null,null,null,null,null,2,2,2,2,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""maplemage""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,2,2,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [18]:
df_pivot = df_pivot.unique(subset=['user_id'])

In [19]:
df_pivot.shape

(73757, 3683)

In [20]:
users.head()

user_id,playtime_forever,playtime_2weeks,item_id,item_name,__index_level_0__
str,i64,i64,i32,str,i64
"""76561197970982…",6,0,10,"""Counter-Strike…",0
"""76561197970982…",0,0,20,"""Team Fortress …",1
"""76561197970982…",7,0,30,"""Day of Defeat""",2
"""76561197970982…",0,0,40,"""Deathmatch Cla…",3
"""76561197970982…",0,0,50,"""Half-Life: Opp…",4


In [21]:
users = users[['user_id', 'playtime_forever', 'playtime_2weeks']]
users_grouped = users.groupby('user_id').agg([
    pl.col('playtime_forever').sum().alias('total_playtime_forever'),
    pl.col('playtime_2weeks').sum().alias('total_playtime_2weeks')
])

In [22]:
users_grouped.head()

user_id,total_playtime_forever,total_playtime_2weeks
str,i64,i64
"""76561198054662…",203383,4135
"""76561198068225…",113908,3350
"""jaguarpanda""",86800,323
"""400primetopkek…",15587,172
"""vintageale""",55784,0


In [23]:
users_grouped.filter(users_grouped['total_playtime_forever'] == 0)

user_id,total_playtime_forever,total_playtime_2weeks
str,i64,i64
"""76561198069701…",0,0
"""76561198083407…",0,0
"""76561198047203…",0,0
"""76561198072663…",0,0
"""76561198081164…",0,0
"""76561198081178…",0,0
"""76561198077908…",0,0
"""BuTcHiouroshi""",0,0
"""HeoSuaXongDzol…",0,0


In [26]:
#df_pivot = df_pivot.cast(pl.UInt8, strict=False)
df_pivot = df_pivot.fill_null(1)

In [27]:
df_final = df_pivot.join(users_grouped, on= 'user_id', how= 'left')

In [28]:
df_final = df_final.fill_null(0)

In [29]:
df_final.head()

user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,…,225833,378660,242590,339910,61730,259410,318520,319970,378580,420930,349740,254130,274350,381020,358260,358340,358290,256500,431840,385890,203250,348670,385630,385980,369000,294230,367780,305920,306040,414390,307130,209120,220090,262850,431510,total_playtime_forever,total_playtime_2weeks
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i64,i64
"""76561198090457…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3180,0
"""76561198091690…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7354,0
"""JakDaRippa""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,23959,791
"""Iwannahoneyboo…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,67222,39
"""76561198003315…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,6812,0


In [30]:
df_final.filter(df_final['total_playtime_forever'] == 0)

user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,…,225833,378660,242590,339910,61730,259410,318520,319970,378580,420930,349740,254130,274350,381020,358260,358340,358290,256500,431840,385890,203250,348670,385630,385980,369000,294230,367780,305920,306040,414390,307130,209120,220090,262850,431510,total_playtime_forever,total_playtime_2weeks
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i64,i64
"""Red_M""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""76561198088873…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""76561197990979…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""76561198139811…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""76561198075234…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""76561198044664…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""76561198026460…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""45979""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
"""mummbles1991""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0


In [31]:
df_final = df_final.with_columns([
    pl.col('total_playtime_forever') / 60,
    pl.col('total_playtime_2weeks') / 60
])

In [32]:
df_final.filter(df_final['total_playtime_forever'] == 0)

user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,…,225833,378660,242590,339910,61730,259410,318520,319970,378580,420930,349740,254130,274350,381020,358260,358340,358290,256500,431840,385890,203250,348670,385630,385980,369000,294230,367780,305920,306040,414390,307130,209120,220090,262850,431510,total_playtime_forever,total_playtime_2weeks
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f64,f64
"""Red_M""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""76561198088873…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""76561197990979…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""76561198139811…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""76561198075234…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""76561198044664…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""76561198026460…",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""45979""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0
"""mummbles1991""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0,0.0


In [33]:
df_final.null_count()

user_id,1250,22200,43110,251610,227300,239030,248820,370360,237930,263360,107200,224500,250320,20920,204100,224600,207610,108710,211420,211820,730,204300,249130,550,65800,220,72850,440,252950,304930,4000,42910,232090,570,218620,224260,…,225833,378660,242590,339910,61730,259410,318520,319970,378580,420930,349740,254130,274350,381020,358260,358340,358290,256500,431840,385890,203250,348670,385630,385980,369000,294230,367780,305920,306040,414390,307130,209120,220090,262850,431510,total_playtime_forever,total_playtime_2weeks
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
pl.DataFrame.write_parquet(df_final, 'users_item_features.parquet')